In [1]:
import pandas as pd
import scanpy as sc
from scipy.spatial import cKDTree
from tqdm import tqdm
from pandas.api.types import CategoricalDtype
from scipy import sparse
import matplotlib.pyplot as plt
import anndata as ad
import numpy as np
import os
import time
from scipy.spatial import ConvexHull
from shapely.geometry import Point, Polygon
from shapely.prepared import prep

import warnings
warnings.filterwarnings('ignore')

In [ ]:
raw_adata = sc.read_h5ad('/mnt/Data16Tc/home/haichao/code/SpaCon/Data/N_20231213_zxw/mouse_1/adata_processed.h5ad')

In [2]:
# path = '/mnt/Data18Td/Data/haichao/merfish_raw_data_zxw1/'
# decode_files_name = os.listdir(path + 'decode/')
# section_dict = {}
# for f in decode_files_name:
#     cellpose = pd.read_csv(path + 'cell_boundary/' + f[6:], index_col=0)
#     adata_tmp = raw_adata[raw_adata.obs.index.isin(list(set(cellpose.index) & set(raw_adata.obs.index)))]
#     print(f"{f}: {adata_tmp.obs['brain_section_label'].unique()}")
#     section_dict[f] = adata_tmp.obs['brain_section_label'].unique()

section_dict = {
 'spots_220720_co2_9_merfish4_adaptor.csv': ['Zhuang-ABCA-1.047', 'Zhuang-ABCA-1.057',  'Zhuang-ABCA-1.067'],
 'spots_220516_wb3_co2_7_5z18R_merfish5.csv': ['Zhuang-ABCA-1.043',  'Zhuang-ABCA-1.053',  'Zhuang-ABCA-1.063'],
 'spots_220708_co2_17B_merfish4_adaptor.csv': ['Zhuang-ABCA-1.104',  'Zhuang-ABCA-1.114'],
 'spots_220726_co2_1B_merfish4_adaptor.csv': ['Zhuang-ABCA-1.012',  'Zhuang-ABCA-1.022',  'Zhuang-ABCA-1.032'],
 'spots_220616_co2_8B_merfish4_adaptor.csv': ['Zhuang-ABCA-1.046',  'Zhuang-ABCA-1.056',  'Zhuang-ABCA-1.066'],
 'spots_220507_wb3_co2_11_5z18R_merfish5.csv': ['Zhuang-ABCA-1.072',  'Zhuang-ABCA-1.082'],
 'spots_220623_co2_4B_merfish4_adaptor.csv': ['Zhuang-ABCA-1.018',  'Zhuang-ABCA-1.028',  'Zhuang-ABCA-1.038'],
 'spots_220530_wb3_co2_4_5z18R_merfish5.csv': ['Zhuang-ABCA-1.037',  'Zhuang-ABCA-1.017',  'Zhuang-ABCA-1.027'],
 'spots_220518_wb3_co2_17_5z18R_merfish5.csv': ['Zhuang-ABCA-1.103',  'Zhuang-ABCA-1.113',  'Zhuang-ABCA-1.093'],
 'spots_220623_wb3_co2_24_5z18R_merfish6.csv': ['Zhuang-ABCA-1.128',  'Zhuang-ABCA-1.136'],
 'spots_220530_wb3_co2_5_5z18R_merfish6.csv': ['Zhuang-ABCA-1.019',  'Zhuang-ABCA-1.029',  'Zhuang-ABCA-1.039'],
 'spots_220620_wb3_co2_B_18_5z18R_merfish5.csv': ['Zhuang-ABCA-1.096',  'Zhuang-ABCA-1.106',  'Zhuang-ABCA-1.116'],
 'spots_220620_wb3_co2_B_22_5z18R_merfish6.csv': ['Zhuang-ABCA-1.124',  'Zhuang-ABCA-1.132',  'Zhuang-ABCA-1.140'],
 'spots_220630_wb3_co2_20_5z18R_merfish6.csv': ['Zhuang-ABCA-1.100',  'Zhuang-ABCA-1.120',  'Zhuang-ABCA-1.110'],
 'spots_220717_co2_13B_merfish4_adaptor.csv': ['Zhuang-ABCA-1.086',  'Zhuang-ABCA-1.076'],
 'spots_220723_co2_6B_merfish4_adaptor.csv': ['Zhuang-ABCA-1.052',  'Zhuang-ABCA-1.062',  'Zhuang-ABCA-1.042'],
 'spots_220503_co2_13_merfish4_adaptor_5z18r.csv': ['Zhuang-ABCA-1.085'],
 'spots_220516_wb3_co2_8_5z18R_merfish6.csv': ['Zhuang-ABCA-1.045',  'Zhuang-ABCA-1.055',  'Zhuang-ABCA-1.065'],
 'spots_220525_wb3_co2_22_5z18R2bd_merfish5.csv': ['Zhuang-ABCA-1.123',  'Zhuang-ABCA-1.131',  'Zhuang-ABCA-1.139'],
 'spots_220605_co2_11B_merfish4_adaptor.csv': ['Zhuang-ABCA-1.071',  'Zhuang-ABCA-1.081'],
 'spots_220601_wb3_co2_2_5z18R2bd_merfish6.csv': ['Zhuang-ABCA-1.033',  'Zhuang-ABCA-1.013',  'Zhuang-ABCA-1.023'],
 'spots_220528_wb3_co2_24_5z18R2bd_merfish6.csv': ['Zhuang-ABCA-1.127',  'Zhuang-ABCA-1.135'],
 'spots_220518_wb3_co2_16_5z18R_merfish6.csv': ['Zhuang-ABCA-1.091',  'Zhuang-ABCA-1.111',  'Zhuang-ABCA-1.101'],
 'spots_220722_wb3_co2_B_3_5z18R_merfish5.csv': ['Zhuang-ABCA-1.036',  'Zhuang-ABCA-1.016',  'Zhuang-ABCA-1.026'],
 'spots_220711_co2_7B_merfish4_adaptor.csv': ['Zhuang-ABCA-1.044',  'Zhuang-ABCA-1.064',  'Zhuang-ABCA-1.054'],
 'spots_220626_co2_14B_merfish4_adaptor.csv': ['Zhuang-ABCA-1.088',  'Zhuang-ABCA-1.078'],
 'spots_220623_wb3_co2_21_5z18R_merfish5.csv': ['Zhuang-ABCA-1.122',  'Zhuang-ABCA-1.130',  'Zhuang-ABCA-1.138'],
 'spots_220513_wb3_co2_9_5z18R_merfish5.csv': ['Zhuang-ABCA-1.048'  'Zhuang-ABCA-1.058',  'Zhuang-ABCA-1.068'],
 'spots_220703_co2_10B_merfish4_adaptor.csv': ['Zhuang-ABCA-1.050',  'Zhuang-ABCA-1.060',  'Zhuang-ABCA-1.070'],
 'spots_220616_wb3_co2_B_16_5z18R_merfish5.csv': ['Zhuang-ABCA-1.102',  'Zhuang-ABCA-1.092'],
 'spots_220706_co2_5B_merfish4_adaptor.csv': ['Zhuang-ABCA-1.010',  'Zhuang-ABCA-1.030',  'Zhuang-ABCA-1.040',  'Zhuang-ABCA-1.020'],
 'spots_220511_co2_6_merfish4_adaptor.csv': ['Zhuang-ABCA-1.051',  'Zhuang-ABCA-1.061',  'Zhuang-ABCA-1.041'],
 'spots_220504_wb3_co2_14_5z18R_merfish5.csv': ['Zhuang-ABCA-1.077',  'Zhuang-ABCA-1.087'],
 'spots_220525_wb3_co2_21_5z18R2bd_merfish6.csv': ['Zhuang-ABCA-1.121',  'Zhuang-ABCA-1.129',  'Zhuang-ABCA-1.137'],
 'spots_220604_wb3_co2_3_5z18R2bd_merfish5.csv': ['Zhuang-ABCA-1.015',  'Zhuang-ABCA-1.025',  'Zhuang-ABCA-1.035'],
 'spots_220517_co2_19_merfish4_adaptor.csv': ['Zhuang-ABCA-1.097',  'Zhuang-ABCA-1.107',  'Zhuang-ABCA-1.117'],
 'spots_220523_co2_23_merfish4_adaptor.csv': ['Zhuang-ABCA-1.125',  'Zhuang-ABCA-1.133',  'Zhuang-ABCA-1.141'],
 'spots_220726_wb3_co2_B_2_5z18R_merfish5.csv': ['Zhuang-ABCA-1.034',  'Zhuang-ABCA-1.014',  'Zhuang-ABCA-1.024'],
 'spots_220606_wb3_co2_12_5z18R2bd_merfish5.csv': ['Zhuang-ABCA-1.073',  'Zhuang-ABCA-1.083'],
 'spots_220501_wb3_co2_15_5z18R_merfish5.csv': ['Zhuang-ABCA-1.089',  'Zhuang-ABCA-1.079'],
 'spots_220523_wb3_co2_18_5z18R2bd_merfish6.csv': ['Zhuang-ABCA-1.095',  'Zhuang-ABCA-1.105',  'Zhuang-ABCA-1.115'],
 'spots_220719_wb3_co2_B_23_5z18R_merfish5.csv': ['Zhuang-ABCA-1.126',  'Zhuang-ABCA-1.134'],
 'spots_220629_co2_15B_merfish4_adaptor.csv': ['Zhuang-ABCA-1.090',  'Zhuang-ABCA-1.080'],
 'spots_220523_wb3_co2_20_5z18R2bd_merfish5.csv': ['Zhuang-ABCA-1.099',  'Zhuang-ABCA-1.109',  'Zhuang-ABCA-1.119'],
 'spots_220601_wb3_co2_1_5z18R2bd_merfish5.csv': ['Zhuang-ABCA-1.031',  'Zhuang-ABCA-1.011',  'Zhuang-ABCA-1.021'],
 'spots_220510_wb3_co2_12_5z18R_merfish5.csv': ['Zhuang-ABCA-1.084',  'Zhuang-ABCA-1.074'],
 'spots_220714_co2_19B_merfish4_adaptor.csv': ['Zhuang-ABCA-1.098',  'Zhuang-ABCA-1.108',  'Zhuang-ABCA-1.118']
 }

In [ ]:
for k in section_dict.keys():
    # for k, v in section_dict.items():
    #     if section in v:
    #         break
    print(k)
    if len(section_dict[k]) == 0:
        continue
    path = '/mnt/Data18Td/Data/haichao/merfish_raw_data_zxw1/'
    decode = pd.read_csv(path + f'decode/{k}')
    decode_list = []
    for z in range(5):
        decode_list.append(decode[decode['global_z'] == z])
    decode = pd.concat(decode_list)

    label_list = []
    for z in range(5):
        label_tmp = pd.read_csv(path + f'/RNA_assign_result/z{z}_{k}')
        label_list.append(label_tmp)
    label = pd.concat(label_list)

    decode['cell_id'] = label['cell_id'].values

    in_cell = decode[(decode['cell_id'] != '-1') & (decode['cell_id'].isin(raw_adata.obs_names))]
    cell_mean_xy = in_cell.groupby('cell_id').agg({
        'global_x': 'mean',
        'global_y': 'mean'
    })
    plt.scatter(cell_mean_xy['global_x'], cell_mean_xy['global_y'], s=1)
    plt.show()
    out_cell_rna = decode[decode['cell_id'] == '-1']
    out_cell_rna = out_cell_rna[~out_cell_rna['target_gene'].str.startswith('blank-')]
    print('raw out_cell_RNA:')
    plt.scatter(out_cell_rna['global_x'], out_cell_rna['global_y'], s=0.1)
    plt.show()

    # start_time = time.time()
    coordinate = cell_mean_xy.values
    hull = ConvexHull(coordinate)
    polygon = Polygon(coordinate[hull.vertices])
    prepared_polygon = prep(polygon)
    min_x = coordinate[:, 0].min()
    max_x = coordinate[:, 0].max()
    min_y = coordinate[:, 1].min()
    max_y = coordinate[:, 1].max()
    out_cell_rna = out_cell_rna[(out_cell_rna['global_x'] >= min_x) & (out_cell_rna['global_x'] <= max_x) & (out_cell_rna['global_y'] >= min_y) & (out_cell_rna['global_y'] <= max_y)]
    points_to_check = np.array(out_cell_rna[['global_x', 'global_y']])
    in_hull = np.array([prepared_polygon.contains(Point(point)) for point in points_to_check])
    out_cell_rna['in_hull'] = in_hull.astype(int)
    out_cell_rna = out_cell_rna[out_cell_rna['in_hull'] == 1]
    print('after filter out_cell_RNA:')
    plt.scatter(out_cell_rna['global_x'], out_cell_rna['global_y'], s=0.1)
    plt.show()
    # end_time = time.time()
    # print(f"time: {end_time - start_time}")

    coordinate = cell_mean_xy.values
    out_cell_coordinate = out_cell_rna[['global_x', 'global_y']].values
    kdtree = cKDTree(coordinate)
    idx = []
    for coord in tqdm(out_cell_coordinate):
        # Query recent neighbor indexes
        _, nearest_index = kdtree.query(coord)
        idx.append(nearest_index)
    out_cell_rna['cell_name'] = list(cell_mean_xy.index[idx])

    # Remove RNA that is too far from the cells from the cell
    # start_time = time.time()
    out_cell_rna.rename(columns={'global_x': 'global_x_out', 'global_y': 'global_y_out'}, inplace=True)
    out_cell_rna = pd.merge(out_cell_rna, cell_mean_xy, left_on='cell_name', right_index=True, how='left')
    out_cell_rna['distance'] = np.sqrt((out_cell_rna['global_x_out'] - out_cell_rna['global_x']) ** 2 + (out_cell_rna['global_y_out'] - out_cell_rna['global_y']) ** 2)
    threshold = out_cell_rna['distance'].quantile(0.30)
    print(f'distance threshold:{threshold}')
    out_cell_rna = out_cell_rna[out_cell_rna['distance'] <= threshold]
    # print('after filter out_cell_RNA:')
    # plt.scatter(out_cell_rna['global_x'], out_cell_rna['global_y'], s=0.1)
    # plt.show()
    # end_time = time.time()
    # print(f"time: {end_time - start_time}")

    bininfo_exclude = out_cell_rna.groupby(['target_gene', 'cell_name']).size().reset_index(name='umi_count')
    out_cell_mean_xy = out_cell_rna.groupby(['cell_name'])[['x', 'y']].mean().reset_index()  # [3587 rows x 2 columns]

    obs = bininfo_exclude['cell_name'].unique()      
    var = bininfo_exclude['target_gene'].unique()
    shape = (len(obs), len(var))
    obs_cat = CategoricalDtype(categories=sorted(obs), ordered=True) 
    var_cat = CategoricalDtype(categories=sorted(var), ordered=True) 
    obs_index = bininfo_exclude["cell_name"].astype(obs_cat).cat.codes   
    var_index = bininfo_exclude["target_gene"].astype(var_cat).cat.codes  
    coo = sparse.coo_matrix((bininfo_exclude["umi_count"], (obs_index, var_index)), shape=shape)
    expcounts = coo.tocsr()

    # to anndata
    adata = ad.AnnData(expcounts)
    adata.obs_names = obs_cat.categories
    adata.var_names = var_cat.categories
    adata.obsm["spatial"] = out_cell_mean_xy[['x', 'y']].values
    adata.obs['totalRNA'] = adata.X.sum(1)

    adata.obs[['brain_section_label', 'x', 'y', 'z']] = raw_adata.obs.loc[adata.obs.index][['brain_section_label', 'x', 'y', 'z']]
    plt.show()
    plt.scatter(adata.obs['z'], adata.obs['y'], s=1, c=adata.obs['totalRNA'])

    for i in range(len(section_dict[k])):
        tmp = adata[adata.obs['brain_section_label'] == section_dict[k][i]]
        plt.scatter(tmp.obs['z'], tmp.obs['y'], s=1, c=tmp.obs['totalRNA'])
        plt.show()
        tmp.write_h5ad(f'/mnt/Data18Td/Data/haichao/merfish_raw_data_zxw1/out_cell_adata/adata_out_cell_distance_q0.3/{section_dict[k][i]}.h5ad')